In [2]:
!pip install pymatreader

  Using cached pymatreader-0.0.30-py3-none-any.whl (9.0 kB)


In [61]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import *
from sklearn import metrics
from pymatreader import read_mat
import numpy as np

In [4]:
data = []
for i in range(10): # channel x T x participant
    data.append( read_mat('song{}_Imputed.mat'.format(i + 21))['data{}'.format(i + 21)] )

In [6]:
data[0].shape

(125, 34795, 20)

In [9]:
data = np.zeros(shape=(10,125,30000,20)) #song x chan x T x person
for i in range(10): # channel x T x participant
    data[i] = read_mat('song{}_Imputed.mat'.format(i + 21))['data{}'.format(i + 21)][:,:30000,:]

In [13]:
data_chanavg = np.mean(data, axis=1) #song x T x person

(10, 125, 30000, 20)


(10, 30000, 20)

In [22]:
ratings = read_mat('behavioralRatings.mat')['behavioralRatings'][:,:,1].T #song x person

In [23]:
ratings.shape

(10, 20)

In [27]:
!pip install surprise

  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp39-cp39-win_amd64.whl size=709737 sha256=c1ad9620f8f9ac37b89a7b194489bf6e2b7ba8dc669076cc94b9837bb59bc0cc
  Stored in directory: c:\users\junom\appdata\local\pip\cache\wheels\6b\10\c9\7f607c8cb522ef378844f41e63b30d7181a6495d2c1ae514e9
Successfully built scikit-surprise


In [29]:
print(data_chanavg.shape)
ratings.shape

(10, 30000, 20)


(10, 20)

In [33]:
pip install einops

Note: you may need to restart the kernel to use updated packages.


In [35]:
from einops import rearrange

In [38]:
dataflat=rearrange(data_chanavg,'s t u -> (s u) t')
ratingsflat = ratings.reshape(200)

(200, 30000)

In [31]:
'''matr = np.array(((1, 2),
              (3, 4)),
                      [[5, 6],
                       [7, 8]])
print(matr.shape)
print(np.reshape(matr, (2, 2, 2)))'''

TypeError: Field elements must be 2- or 3-tuples, got '[5, 6]'

In [62]:
clf = RandomForestRegressor()
x_train, x_test, y_train, y_test = train_test_split(dataflat, ratingsflat, test_size = 0.70, random_state = 5)
clf.fit(x_train, y_train)

RandomForestRegressor()

In [63]:
y_pred_test = clf.predict(x_test)
diffs = []
for i in range(50):
    diffs.append( y_pred_test[i] - y_test[i] )
print(np.mean([x**2 for x in diffs])) #could be worse..

4.181856000000001


In [78]:
import random
print(np.mean([(random.uniform(1,10) - random.uniform(1,10))**2 for _ in range(1000)]))

13.181604771511768


In [79]:
clf = AdaBoostRegressor()
clf.fit(x_train, y_train)
y_pred_test = clf.predict(x_test)
diffs = []
for i in range(50):
    diffs.append( y_pred_test[i] - y_test[i] )
print(np.mean([x**2 for x in diffs]))

4.303830811865886


In [80]:
clf = GradientBoostingRegressor()
clf.fit(x_train, y_train)
y_pred_test = clf.predict(x_test)
diffs = []
for i in range(50):
    diffs.append( y_pred_test[i] - y_test[i] )
print(np.mean([x**2 for x in diffs]))

4.679148399125711


In [83]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
clf.fit(x_train, y_train)
y_pred_test = clf.predict(x_test)
diffs = []
for i in range(50):
    diffs.append( y_pred_test[i] - y_test[i] )
print(np.mean([x**2 for x in diffs]))

5.24


In [85]:
from sklearn import neighbors
clf = neighbors.KNeighborsRegressor()
clf.fit(x_train, y_train)
y_pred_test = clf.predict(x_test)
diffs = []
for i in range(50):
    diffs.append( y_pred_test[i] - y_test[i] )
print(np.mean([x**2 for x in diffs]))

4.2304
